In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [2]:
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
tf.keras.mixed_precision.experimental.set_policy(policy) 

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  GeForce GTX 1060, compute capability 6.1
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [3]:
tokenizer = Tokenizer()

data = open('compile.txt').read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [5]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=10, verbose=1)
#print model.summary()
print(model)


  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)

Epoch 1/10
1232/1232 [==============================] - 105s 77ms/step - loss: 6.5243 - accuracy: 0.0682
Epoch 2/10
1232/1232 [==============================] - 97s 79ms/step - loss: 4.5067 - accuracy: 0.2457
Epoch 3/10
1232/1232 [==============================] - 122s 99ms/step - loss: 3.3562 - accuracy: 0.3553
Epoch 4/10
1232/1232 [==============================] - 111s 90ms/step - loss: 2.6432 - accuracy: 0.4407
Epoch 5/10
1232/1232 [==============================] - 122s 99ms/step - loss: 2.2085 - accuracy: 0.5094
Epoch 6/10
1232/1232 [==============================] - 118s 96ms/step - loss: 1.9332 - accuracy: 0.5517
Epoch 7/10
1232/1232 [==============================] - 124s 100ms/step - loss: 1.8151 - accuracy: 0.5743
Epoch 8/10
1232/1232 [==============================] - 119s 96ms/step - loss: 1.6619 - accuracy: 0.6047
Epoch 9/10
1232/1232 [==============================] - 119s 97ms/step - loss: 1.6019 - a

In [6]:
seed_text = "I'm very gay and very jesus please love me thank you"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

c:\users\orion\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


I'm very gay and very jesus please love me thank you but i'm not about to expect something more than you got me to be discovered love me in my heart and i feel just like more my own way i feel that empty outline changed my best i can feel the rhythm of your heart de vo tion x4 and have no shit wait the flood valentine spurn my brother's daughter i be invited to the world had it anyway sometimes it may not look just like me like a trumpet sound and i will touch of me i have no idea if i could put up on your plate
